# 인터파크 베스트셀러 크롤링

In [8]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

#### 1. 데이터 가져오기

In [9]:
site = 'http://book.interpark.com'
url = site + '/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller'
res = requests.get(url)

In [10]:
soup = BeautifulSoup(res.text, 'html.parser')

#### 2. 원하는 데이터 찾기

In [11]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

#### 3. 하나의 데이터로 원하는 정보 찾기

In [12]:
li = lis[9]
href = li.select_one('.coverImage').find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354628618&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [13]:
# 책 제목
title = li.select_one('.itemName').get_text().strip()
title

'머니 체인저'

In [14]:
# 저자
author = li.select_one('.author').get_text().strip()
author

'문준희'

In [15]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'마인드셋(Mindset)'

In [16]:
# 가격
price = li.select_one('.price > em').get_text()
price = int(price.replace(',',''))
price

14400

In [17]:
# 순위: 1위 ~ 9위
li.select_one('.rankBtn_ctrl')

<span class="rankBtn_ctrl rkNum_M01"></span>

In [18]:
li.select_one('.rankBtn_ctrl')['class']

['rankBtn_ctrl', 'rkNum_M01']

In [19]:
li.select_one('.rankBtn_ctrl')['class'][1]

'rkNum_M01'

In [20]:
li.select_one('.rankBtn_ctrl')['class'][1][-1]

'1'

In [21]:
rank = li.select_one('.rankBtn_ctrl')['class'][1][-1]
rank

'1'

- 10위 ~ 15위

In [22]:
element = '<div class="rankNumber digit2"><span class="rankBtn_ctrl rkNum_M01 "></span><span class="rankBtn_ctrl rkNum_M00 "></span>           </div>'
mini_soup = BeautifulSoup(element, 'html.parser')

In [23]:
mini_soup.select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M00"></span>]

- 모든 경우에 대처

In [24]:
rank_data = li.select('.rankBtn_ctrl')
if len(li.select('.rankBtn_ctrl')) == 1:
    rank = int(rank_data[0]['class'][1][-1])
else:
    rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
    rank = int(rank)
rank

10

- Sub page

In [25]:
url = site + href
res = requests.get(url)
sub_soup = BeautifulSoup(res.text, 'html.parser')

In [26]:
sub_lis = sub_soup.select('.bInfo_txt > li')
len(sub_lis)

5

In [27]:
for li in sub_lis:
    print(li.get_text())

저 : 문준희
출판사 : 마인드셋(Mindset)
발행 : 2022년 04월 11일
쪽수 : 284
ISBN : 9791191043716


In [33]:
trans = '-'
for li in sub_lis:
    text = li.get_text().strip()
    if '발행 :' in text:
        date = text.split(':')[1].strip()
        date = re.sub('[^0-9]','', date)            # 정규표현식
    elif '쪽수 :' in text:
        page = int(text.split(':')[1].strip())
    elif '역 :' in text:
        trans = text.split(':')[1].strip()
    elif '저 :' in text:
        author = text.split(':')[1].strip()
print(date, page, trans, author)

20211221 392 - 최태성


#### 4. 페이지에 있는 모든 데이터 추출

In [29]:
from tqdm.notebook import tqdm

In [30]:
lines = []
for li in tqdm(lis):
    href = li.select_one('.coverImage').find('a')['href']
    title = li.select_one('.itemName').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em').get_text()
    price = int(price.replace(',',''))
    rank_data = li.select('.rankBtn_ctrl')
    if len(li.select('.rankBtn_ctrl')) == 1:
        rank = int(rank_data[0]['class'][1][-1])
    else:
        rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
        rank = int(rank)

    url = site + href
    res = requests.get(url)
    sub_soup = BeautifulSoup(res.text, 'html.parser')
    sub_lis = sub_soup.select('.bInfo_txt > li')
    trans = '-'
    for li in sub_lis:
        text = li.get_text().strip()
        if '발행 :' in text:
            date = text.split(':')[1].strip()
            date = re.sub('[^0-9]','', date)            # 정규표현식
        elif '쪽수 :' in text:
            page = int(text.split(':')[1].strip())
        elif '역 :' in text:
            trans = text.split(':')[1].strip()
        elif '저 :' in text:
            author = text.split(':')[1].strip()

    lines.append([rank, title, author, trans, company, page, date, price])


  0%|          | 0/15 [00:00<?, ?it/s]

#### 5. 데이터프레임 만들고 저장하기

In [31]:
df = pd.DataFrame(lines, columns='순위 제목 저자 역자 출판사 쪽수 발행일 가격'.split())
df

,순위,제목,저자,역자,출판사,쪽수,발행일,가격
0,1,저주토끼,정보라,-,아작,328,20220401,13320
1,2,긴긴밤,루리,-,문학동네,144,20210203,10350
2,3,웰씽킹(10만 부 기념 한정판 골드 에디션),켈리 최,-,다산북스,316,20211110,14400
3,3,럭키(10만부 기념 황금열쇠 양장 특별판),김도윤,-,북로망스,280,20210825,14400
4,5,꽃샘바람에 흔들린다면 너는 꽃,류시화,-,수오서재,160,20220411,12150
5,5,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,-,나무옆의자,268,20210420,12600
6,7,김학렬의 부동산 투자 절대 원칙,김학렬(빠숑),-,에프엔미디어,540,20220412,22500
7,7,파이어 FIRE,강환국,-,페이지2북스,428,20220408,16200
8,7,파이어족 강환국의 하면 된다! 퀀트 투자,강환국,-,에프엔미디어,392,20210915,16830
9,10,머니 체인저,문준희,-,마인드셋(Mindset),284,20220411,14400


In [32]:
df.to_csv('인터파크.csv', index=False)